# 🤖 03_Modeling — 모델링 및 해석

## 목적
- Data Leakage 피처 제거 후 모델 학습
- Logistic Regression / Random Forest / Gradient Boosting 비교
- 하이퍼파라미터 튜닝 (GridSearchCV)
- Feature Importance 분석
- 비즈니스 제언 도출

## 1. 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (classification_report, roc_auc_score, roc_curve,
                             confusion_matrix, f1_score)
import warnings
warnings.filterwarnings('ignore')

## 2. 데이터 로드 & Data Leakage 제거

In [ ]:
df = pd.read_csv('../data/model_dataset.csv')

# Data Leakage 피처 제거
# 이유: 방문 횟수/최대 도달 주차는 탈락의 "결과"이지 "원인"이 아님
leakage_cols = ['VISIT_TOTAL', 'VISIT_SCHEDULED', 'VISIT_UNSCHEDULED', 'MAX_WEEK', 'CM_COUNT']
df_model = df.drop(columns=leakage_cols + ['USUBJID'])

features = [c for c in df_model.columns if c != 'DROPOUT']
print(f"모델링 피처: {len(features)}개")
for i, f in enumerate(features, 1):
    print(f"  {i:2d}. {f}")

## 3. Train/Test Split

In [ ]:
X = df_model[features]
y = df_model['DROPOUT']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=features, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=features, index=X_test.index)

print(f"Train: {X_train.shape[0]}명 (Completed={sum(y_train==0)}, Dropout={sum(y_train==1)})")
print(f"Test:  {X_test.shape[0]}명 (Completed={sum(y_test==0)}, Dropout={sum(y_test==1)})")

## 4. Baseline 모델 비교 (5-Fold CV)

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

models = {
    'Logistic Regression': (LogisticRegression(max_iter=1000, random_state=42), X_train_scaled),
    'Random Forest': (RandomForestClassifier(n_estimators=200, random_state=42), X_train),
    'Gradient Boosting': (GradientBoostingClassifier(n_estimators=200, random_state=42), X_train)
}

for name, (model, X_tr) in models.items():
    scores = cross_val_score(model, X_tr, y_train, cv=cv, scoring='roc_auc')
    print(f"{name}: AUC = {scores.mean():.3f} ± {scores.std():.3f}")

## 5. 하이퍼파라미터 튜닝

In [ ]:
# Gradient Boosting
gb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [2, 3, 4, 5],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_samples_split': [5, 10, 20],
    'subsample': [0.8, 1.0]
}
gb_grid = GridSearchCV(GradientBoostingClassifier(random_state=42),
                       gb_params, cv=cv, scoring='roc_auc', n_jobs=-1)
gb_grid.fit(X_train, y_train)
print(f"GB Best CV AUC: {gb_grid.best_score_:.3f}")
print(f"GB Best Params: {gb_grid.best_params_}")

# Random Forest
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [5, 10, 20],
    'min_samples_leaf': [2, 5, 10]
}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42),
                       rf_params, cv=cv, scoring='roc_auc', n_jobs=-1)
rf_grid.fit(X_train, y_train)
print(f"\nRF Best CV AUC: {rf_grid.best_score_:.3f}")
print(f"RF Best Params: {rf_grid.best_params_}")

# Logistic Regression
lr_params = {'C': [0.001, 0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}
lr_grid = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42),
                       lr_params, cv=cv, scoring='roc_auc', n_jobs=-1)
lr_grid.fit(X_train_scaled, y_train)
print(f"\nLR Best CV AUC: {lr_grid.best_score_:.3f}")
print(f"LR Best Params: {lr_grid.best_params_}")

## 6. Test Set 최종 평가

In [ ]:
tuned_models = {
    'LR': (lr_grid.best_estimator_, X_test_scaled),
    'RF': (rf_grid.best_estimator_, X_test),
    'GB': (gb_grid.best_estimator_, X_test)
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = {'LR': '#3498db', 'RF': '#2ecc71', 'GB': '#e74c3c'}

best_auc, best_name, best_model = 0, '', None

for name, (model, X_t) in tuned_models.items():
    y_prob = model.predict_proba(X_t)[:, 1]
    y_pred = model.predict(X_t)
    auc = roc_auc_score(y_test, y_prob)
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    
    print(f"\n{name}: AUC={auc:.3f}")
    print(classification_report(y_test, y_pred, target_names=['Completed', 'Dropout']))
    
    axes[0].plot(fpr, tpr, color=colors[name], label=f'{name} (AUC={auc:.3f})', linewidth=2)
    if auc > best_auc:
        best_auc, best_name, best_model = auc, name, model
        best_y_prob, best_y_pred = y_prob, y_pred

axes[0].plot([0,1],[0,1], 'k--', alpha=0.3)
axes[0].set_title('ROC Curves', fontweight='bold')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].legend()

cm_matrix = confusion_matrix(y_test, best_y_pred)
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='Blues', ax=axes[1],
            xticklabels=['Completed', 'Dropout'], yticklabels=['Completed', 'Dropout'])
axes[1].set_title(f'Confusion Matrix — {best_name}', fontweight='bold')
axes[1].set_ylabel('Actual'); axes[1].set_xlabel('Predicted')
plt.tight_layout()
plt.savefig('../outputs/figures/roc_confusion.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"\n🏆 Best: {best_name} (AUC={best_auc:.3f})")

## 7. Feature Importance 분석

In [ ]:
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
else:
    importances = np.abs(best_model.coef_[0])

feat_imp = pd.DataFrame({'feature': features, 'importance': importances})
feat_imp = feat_imp.sort_values('importance', ascending=False)

fig, ax = plt.subplots(figsize=(10, 8))
top15 = feat_imp.head(15).sort_values('importance')
colors_fi = ['#e74c3c' if v > feat_imp['importance'].quantile(0.75) else '#3498db' for v in top15['importance']]
ax.barh(range(len(top15)), top15['importance'], color=colors_fi, edgecolor='white')
ax.set_yticks(range(len(top15)))
ax.set_yticklabels(top15['feature'])
ax.set_title('Top 15 Feature Importance', fontweight='bold', fontsize=14)
ax.set_xlabel('Importance')
plt.tight_layout()
plt.savefig('../outputs/figures/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nTop 10 Features:")
for i, (_, row) in enumerate(feat_imp.head(10).iterrows(), 1):
    print(f"  {i:2d}. {row['feature']}: {row['importance']:.4f}")

## 8. 결론 및 비즈니스 제언

### 모델 성능 요약

| 모델 | CV AUC | Test AUC |
|------|--------|----------|
| Logistic Regression | ~0.73 | ~0.63 |
| Random Forest | ~0.73 | ~0.63 |
| Gradient Boosting | ~0.76 | ~0.61 |

> AUC 0.6~0.7 수준은 이 데이터의 특성(254명의 소규모 데이터, leakage 제거 후)을 고려하면 합리적인 결과입니다.

### 핵심 발견

**1. 투약군(ARM)이 가장 강력한 예측 변수**
- Placebo군 탈락률 32.6% vs Xanomeline군 ~70%
- 약물 자체의 부작용이 탈락의 주요 원인

**2. Baseline 활력징후가 탈락 예측에 기여**
- 기저 수축기혈압(SYSBP), 맥박(PULSE), 이완기혈압(DIABP)
- 활력징후가 불안정한 환자일수록 부작용에 더 취약할 가능성

**3. AE 패턴이 중요한 신호**
- MODERATE 이상의 AE, 약물 관련 AE가 탈락과 양의 상관관계
- MILD AE는 오히려 완료군에서 더 많음 (경미한 부작용은 탈락으로 이어지지 않음)

### 비즈니스 제언

1. **투약군 환자 대상 강화된 모니터링 프로그램** 도입
   - 특히 피부 관련 부작용 발생 시 즉각적인 대응 프로토콜

2. **Baseline 활력징후 기반 위험 계층화**
   - 혈압/맥박이 불안정한 환자에게 사전 상담 및 관리 강화

3. **AE 심각도 기반 조기 경고 시스템**
   - MODERATE 이상 AE 발생 시 자동 알림 + 추가 방문 스케줄링

### 한계점 및 향후 과제

- **소규모 데이터(254명)**: 더 많은 데이터로 모델 검증 필요
- **Data Leakage 제거 후 성능 하락**: 실무에서는 초기 방문 데이터(첫 2~4주)만으로 예측하는 모델이 더 실용적
- **시간적 요소 미반영**: 초기 몇 주간의 AE 패턴으로 탈락을 예측하는 시계열 접근 고려